This notebook contains code that performs a dynamic sentiment analysis based on emojis.

Emojis are extracted from tweets, and the tweet text is used to predict the emoji.

In [1]:
from extract_tweets import get_raw_tweets
import pickle

raw_tweets = get_raw_tweets('BackLAOut',1e5)
raw_tweets += get_raw_tweets('BackNYOut',1e5)

with open('raw_BackLA_BackNY', 'wb') as fp:
    pickle.dump(raw_tweets, fp)

10000 of 100000.0 tweets read
20000 of 100000.0 tweets read
30000 of 100000.0 tweets read
40000 of 100000.0 tweets read
50000 of 100000.0 tweets read
60000 of 100000.0 tweets read
70000 of 100000.0 tweets read
80000 of 100000.0 tweets read
90000 of 100000.0 tweets read
10000 of 100000.0 tweets read
20000 of 100000.0 tweets read
30000 of 100000.0 tweets read
40000 of 100000.0 tweets read
50000 of 100000.0 tweets read
60000 of 100000.0 tweets read
70000 of 100000.0 tweets read
80000 of 100000.0 tweets read
90000 of 100000.0 tweets read


In [2]:
from extract_tweets import clean_tweets
import pickle
with open('raw_BackLA_BackNY', 'rb') as fp:
    raw_tweets = [tweet for tweet in pickle.load(fp)]
    
cleaned_tweets = clean_tweets(raw_tweets)

with open('clean_BackLA_BackNY', 'wb') as fp:
    pickle.dump(cleaned_tweets, fp)
    
len(cleaned_tweets)

10.0% read
20.0% read
30.0% read
40.0% read
50.0% read
60.0% read
70.0% read
80.0% read
90.0% read
100.0% read


25262

In [3]:
import random
import emoji
from itertools import chain
from collections import OrderedDict
from extract_tweets import get__X_and__y
import pickle
with open('clean_BackLA_BackNY', 'rb') as fp:
    cleaned_tweets = [tweet for tweet in pickle.load(fp)]

random.shuffle(cleaned_tweets)

emojis = emoji.core.unicode_codes.UNICODE_EMOJI
emoji_sentiment1 = dict(zip(emojis, [0]*len(emojis)))
emoji_sentiment2 = \
OrderedDict([('☹', -1),('👺', -1),('👿', -1),('💔', -1),('💙', -1),('🖤', -1),('😑', -1),
             ('😒', -1),('😓', -1),('😔', -1),('😕', -1),('😖', -1),('😞', -1),('😟', -1),
             ('😠', -1),('😡', -1),('😢', -1),('😣', -1),('😤', -1),('😥', -1),('😦', -1),
             ('😧', -1),('😨', -1),('😩', -1),('😫', -1),('😭', -1),('😰', -1),('😱', -1),
             ('😲', -1),('😳', -1),('😵', -1),('😾', -1),('😿', -1),('🙁', -1),('🙄', -1),
             ('🤒', -1),('🤕', -1),('🤢', -1),('🤥', -1),('🤬', -1),('🤮', -1),('🥵', -1),
             ('🥺', -1),('🤦', -1),('🤦🏻', -1),('🤦🏼', -1),('🤦🏽', -1),('🤦🏾', -1),('🤦🏿', -1),
             
             ('☺',  1),('♥',  1),('❣',  1),('❤',  1),('🐱',  1),('👅',  1),('💑',  1),
             ('💓',  1),('💕',  1),('💖',  1),('💗',  1),('💘',  1),('💚',  1),('💛',  1),
             ('💜',  1),('💝',  1),('💞',  1),('💟',  1),('😀',  1),('😁',  1),('😂',  1),
             ('😃',  1),('😄',  1),('😅',  1),('😆',  1),('😇',  1),('😈',  1),('😉',  1),
             ('😊',  1),('😋',  1),('😌',  1),('😍',  1),('😎',  1),('😏',  1),('😗',  1),
             ('😘',  1),('😙',  1),('😚',  1),('😛',  1),('😜',  1),('😝',  1),('😸',  1),
             ('😺',  1),('😻',  1),('😼',  1),('😽',  1),('🙂',  1),('🙃',  1),('🤍',  1),
             ('🤎',  1),('🤗',  1),('🤠',  1),('🤤',  1),('🤪',  1),('🤭',  1),('🥰',  1),
             ('🥳',  1),('🥴',  1),('🧡',  1)])
for e in emojis:
    if e in emoji_sentiment2:
        del emoji_sentiment1[e]
emoji_sentiment = OrderedDict(chain(emoji_sentiment1.items(),emoji_sentiment2.items()))

with open('emoji_sentiment', 'wb') as fp:
    pickle.dump(emoji_sentiment, fp)

In [4]:
from extract_tweets import get__X_and__y
import pickle
with open('emoji_sentiment', 'rb') as fp:
    emoji_sentiment = pickle.load(fp)

_X, _y = get__X_and__y(cleaned_tweets, emoji_sentiment)

with open('_X', 'wb') as fp:
    pickle.dump(_X, fp)
with open('_y', 'wb') as fp:
    pickle.dump(_y, fp)

In [5]:
from collections import Counter, OrderedDict
from math import ceil
import numpy as np
from functools import reduce
import pickle
with open('_X', 'rb') as fp:
    _X = [tweet for tweet in pickle.load(fp)]
with open('_y', 'rb') as fp:
    _y = pickle.load(fp)

batch_size = 64

def split_tr_val_te(X):
    percent_tr = 1/3
    percent_val = 1/3

    tr_size = batch_size*ceil(percent_tr*len(X)/batch_size)
    val_size = ceil(percent_val*(len(X)-tr_size)/(1-percent_tr))
    te_size = len(X) - tr_size - val_size

    X_val = X[:val_size]
    X_tr = X[val_size:-te_size]
    X_te = X[-te_size:]
    print(len(X),len(X_tr)/len(X),len(X_val)/len(X),len(X_te)/len(X),len(X_tr)/batch_size)
    return X_tr, X_val, X_te

_X_tr,_X_val,_X_te = split_tr_val_te(_X)
_y_tr,_y_val,_y_te = split_tr_val_te(_y)

def split_tweets_into_words(_X):
    return [[word for word in tweet.split(' ')] for tweet in _X]

_X_tr = split_tweets_into_words(_X_tr)
_X_val = split_tweets_into_words(_X_val)
_X_te = split_tweets_into_words(_X_te)

emoji_counts = Counter()
for i in _y_tr:
    emoji_counts[i] += 1
emoji_counts, {k:v/sum(emoji_counts.values()) for k,v in emoji_counts.items()}

24120 0.33432835820895523 0.3328358208955224 0.3328358208955224 126.0
24120 0.33432835820895523 0.3328358208955224 0.3328358208955224 126.0


(Counter({1: 3082, 0: 2559, -1: 2423}),
 {1: 0.3821924603174603, 0: 0.31733630952380953, -1: 0.3004712301587302})

In [6]:
# get word counts and vocabulary size
runonsentence = ' '.join([' '.join(tweet) for tweet in _X_tr]+
                         [' '.join(tweet) for tweet in _X_val]+
                         [' '.join(tweet) for tweet in _X_te]).split(' ')

words = np.unique(runonsentence)
def get_counts_adv(_X):
    d1 = dict(zip(words,[0]*len(words)))
    d2 = dict(zip(words,[0]*len(words)))
    d3 = dict(zip(words,[0]*len(words)))
    
    for x in _X:
        for w in x:
            d1[w] += 1
    m1 = np.mean(list(d1.values()))
    sd1 = np.std(list(d1.values()))
    print(m1,sd1)
    
    for x in _X:
        d = Counter()
        for w in x:
            d[w] = 1
        for w in d:
            d2[w] += d[w]
    m2 = np.mean(list(d2.values()))
    sd2 = np.std(list(d2.values()))
    print(m2,sd2)
    
    for w in set(w for x in _X for w in x):
        d3[w] = (d1[w] - m1) / sd1 + (d2[w] - m2) / sd2

    return d3

wc_tr = get_counts_adv(_X_tr)
wc_val = get_counts_adv(_X_val)
wc_te = get_counts_adv(_X_te)

for w in words:
    if wc_tr[w]*wc_val[w]*wc_te[w]==0:
        del wc_tr[w]
        del wc_val[w]
        del wc_te[w]
        
wc_tr

2.3323501856414173 26.181564719620926
2.215933932475103 22.62925348598156
2.3357160206808008 26.029517011234752
2.2246434643811375 22.66348064654215
2.346334015753496 26.670402956160018
2.2270377181720393 22.99426521483986


{'': 25.434852740146884,
 '0': 0.3896907062675963,
 '04': -0.02223630086857618,
 '1': 1.9932081475876795,
 '10': 1.9550133333850517,
 '100': 1.1311593191127067,
 '1000': 0.1425345019858928,
 '101': -0.10462170229581066,
 '1010': -0.10462170229581066,
 '10k': -0.10462170229581066,
 '11': 0.3896907062675963,
 '1111': -0.10462170229581066,
 '12': 0.3896907062675963,
 '127': 0.3073053048403618,
 '13': 0.5544615091220653,
 '13th': -0.10462170229581066,
 '14': 0.01595851333405176,
 '14th': -0.10462170229581066,
 '15': 1.0869687318881,
 '16': 0.1425345019858928,
 '17': 0.18072931618852073,
 '18': 0.3896907062675963,
 '19': -0.02223630086857618,
 '1st': 0.1425345019858928,
 '2': 4.705930621664439,
 '20': 1.0487739176854722,
 '200': -0.02223630086857618,
 '2016': 0.3073053048403618,
 '2019': -0.02223630086857618,
 '2020': 1.3341249361698035,
 '2021': 0.06014910055865831,
 '21': -0.02223630086857618,
 '22': 0.2249199034131273,
 '23': -0.02223630086857618,
 '24': 0.3073053048403618,
 '247': -0.02

In [7]:
def get_most_frequent_keys(d,n):
    a = np.array(list(d.values()),dtype='float')
    a -= np.nanmin(a)
    N = min(n,len(d))
    inds = a >= 0.99 * min(a[np.argpartition(a,-N)][-N:])
    d = dict(np.array(list(d.items()))[inds])
    for k in d:
        d[k] = float(d[k])
    odct = OrderedDict(sorted(d.items(),key=lambda x:x[1], reverse=True))
    return odct

vocab = get_most_frequent_keys(wc_tr,1000)
print(len(vocab))

def get_counts(i):
    d = Counter()
    for j in i:
        d[j] += 1
    return d

runonsentence = [w if w in vocab else '<UNK>' for w in runonsentence]
word_counts = get_counts(runonsentence)
    
assert(min(list(word_counts.values())) > 1)

word_counts = OrderedDict(sorted(word_counts.items()), key=lambda x: x[0], reverse=True)

vocabulary_size = len(word_counts)
vocabulary_size, word_counts['<UNK>'], _X_tr[:10]

1093


(1096,
 55663,
 [['amp', 'delete', 'everything', 'meg', 'related'],
  ['gn', 'sleep', 'toight'],
  ['tron',
   'i',
   'was',
   'hoping',
   'you',
   'wasn’t',
   'gonna',
   'say',
   'king',
   'henry'],
  ['x'],
  ['thedreamr',
   'lmaooo',
   'i’m',
   'so',
   'high',
   'i',
   'probably',
   'wouldn’t',
   'even',
   'have',
   'noticed'],
  ['dawng', 'za', 'hostel'],
  ['shitpost',
   'when',
   'i',
   'first',
   'played',
   'drifblim',
   'was',
   'my',
   'flying',
   'guy',
   'i',
   'will',
   'always',
   'treasure',
   'him'],
  ['dray1', 'he', 'is', 'blessed'],
  ['i',
   'wonder',
   'if',
   'this',
   'our',
   'last',
   'summer',
   'everything',
   'feel',
   'strange'],
  ['thanks', 'love', '️']])

In [8]:
T=np.array([(0,len(x)) for x in _X_tr]+
           [(1,len(x)) for x in _X_val]+
           [(2,len(x)) for x in _X_te])
T=np.vstack([T.T,np.arange(len(T))]).T
tr=len(_X_tr)
val=len(_X_val)
T[:,2][tr:] -= tr
T[:,2][tr+val:] -= val
_X_tr[0:2],_X_val[0:2],_X_te[0:2],T[0:2],T[tr:tr+2],T[tr+val:tr+val+2]

([['amp', 'delete', 'everything', 'meg', 'related'],
  ['gn', 'sleep', 'toight']],
 [['oop'], ['maybe', 'if', 'i', 'was', 'a', 'kid', 'in', 'school']],
 [['ofozone', 'i’m', 'not', 'admitting', 'nor', 'denying', 'anything'],
  ['iight', 'man', 'it’s', 'time', 'for', 'bed']],
 array([[0, 5, 0],
        [0, 3, 1]]),
 array([[1, 1, 0],
        [1, 8, 1]]),
 array([[2, 7, 0],
        [2, 6, 1]]))

In [9]:
i = 0
for t in T:
    for j in range(t[1]):
        if t[0] == 0:
            _X_tr[t[2]][j] = runonsentence[i]
        elif t[0] == 1:
            _X_val[t[2]][j] = runonsentence[i]
        else:
            _X_te[t[2]][j] = runonsentence[i]
        i += 1
_X_tr[0:2],_X_val[0:2],_X_te[0:2]

([['amp', '<UNK>', 'everything', '<UNK>', '<UNK>'],
  ['<UNK>', 'sleep', '<UNK>']],
 [['<UNK>'], ['maybe', 'if', 'i', 'was', 'a', 'kid', 'in', 'school']],
 [['<UNK>', 'i’m', 'not', '<UNK>', '<UNK>', '<UNK>', 'anything'],
  ['<UNK>', 'man', 'it’s', 'time', 'for', 'bed']])

In [10]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
import pickle
with open('emoji_sentiment', 'rb') as fp:
    emoji_sentiment = pickle.load(fp)

def convert_to_id(_X):
    return [[word2id[word] for word in tweet] for tweet in _X]

def convert_to_one_hot(_y):
    #return enc.transform(np.array(_y).reshape(-1,1)).toarray()
    _y = np.array(_y) - np.array(_y).min()
    _y = _y * (len(np.unique(_y))-1) / max(_y)
    print(_y)
    return to_categorical(_y, num_classes=len(np.unique(list(emoji_sentiment.values()))))

word2id = {}
for i,word in enumerate(word_counts):
    word2id[word] = i

max_words = max([len(x) for x in _X])
X_tr= sequence.pad_sequences(convert_to_id(_X_tr), maxlen=max_words)
X_val = sequence.pad_sequences(convert_to_id(_X_val), maxlen=max_words)
X_te = sequence.pad_sequences(convert_to_id(_X_te), maxlen=max_words)

y_tr = convert_to_one_hot(_y_tr)
y_val = convert_to_one_hot(_y_val)
y_te = convert_to_one_hot(_y_te)

X_tr[:20], y_tr[:20], max_words

[2. 1. 0. ... 1. 0. 2.]
[1. 1. 2. ... 2. 2. 2.]
[1. 1. 0. ... 2. 0. 2.]


(array([[   0,    0,    0, ...,  305,   23,   23],
        [   0,    0,    0, ...,   23,  826,   23],
        [   0,    0,    0, ...,  779,  509,   23],
        ...,
        [   0,    0,    0, ...,  587,  702,  818],
        [   0,    0,    0, ...,  925,   23, 1041],
        [   0,    0,    0, ...,  483,  631,   23]], dtype=int32),
 array([[0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.]], dtype=float32),
 140)

This concludes the data grooming. Run the analysis!

In [11]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                  classes=np.arange(len(y_tr[0])),
                                                  y=np.argmax(y_tr,axis=1))
# class_weights = class_weight.compute_class_weight('balanced',
#                                                   classes=np.arange(len(y_tr[0]))-1,
#                                                   y=_y_tr)

print(np.sum(y_tr,axis=0)*class_weights)

class_weights = {i:w for i,w in zip(range(len(y_tr[0])),class_weights)}

class_weights

[2688. 2688. 2688.]


{0: 1.1093685513825835, 1: 1.0504103165298946, 2: 0.872160934458144}

In [12]:
from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Dense, Dropout, Bidirectional, BatchNormalization
)

def sdlkfdsf(x):
    if x < 10: return x
    return int(x / 10) * 10

embedding_size=32
raw_units = 0.32 * (np.log1p(len(X_tr)) + np.sqrt(3 * vocabulary_size * embedding_size))
units1 = sdlkfdsf(0.5 * raw_units)
units2 = sdlkfdsf(1.0 * raw_units)
units3 = sdlkfdsf(1.5 * raw_units)
units4 = sdlkfdsf(3.0 * raw_units)
print(raw_units,units1,units2,units3,units4)
dropout = 0.2

embed          = Embedding(vocabulary_size, embedding_size, input_length=max_words)

forward_layer  = LSTM(units1, dropout=dropout, recurrent_dropout=dropout,
                      return_sequences=True)
                      
backward_layer = LSTM(units2, dropout=dropout, recurrent_dropout=dropout,
                      return_sequences=True, go_backwards=True,
                      activation='softsign', recurrent_activation='softsign'
                     )
                      
intermed_layer = LSTM(units3, dropout=dropout, recurrent_dropout=dropout)

final_layer    = Dense(units4, activation='relu')

inputs = Input(batch_input_shape=(None, None))
x = embed(inputs)
x = Bidirectional(forward_layer, backward_layer=backward_layer)(x)
x = intermed_layer(x)
x = BatchNormalization()(x)
x = final_layer(x)
x = Dropout(dropout)(x)
outputs = Dense(3, activation='softmax')(x)

model=Model(inputs=inputs, outputs=outputs)

model.summary()

106.67694333247104 50 100 160 320
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 32)          35072     
_________________________________________________________________
bidirectional (Bidirectional (None, None, 150)         69800     
_________________________________________________________________
lstm_2 (LSTM)                (None, 160)               199040    
_________________________________________________________________
batch_normalization (BatchNo (None, 160)               640       
_________________________________________________________________
dense (Dense)                (None, 320)               51520     
_________________________________________________________________
dropout (Dropout)   

In [13]:
loss = 'binary_crossentropy' if len(class_weights) == 2 else 'categorical_crossentropy'
model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = 'temp.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,
                             save_best_only=True, mode='max')
num_epochs = 10
print(len(X_tr))
model.fit(X_tr, y_tr, validation_data=(X_val, y_val), batch_size=batch_size,
          epochs=num_epochs, class_weight=class_weights, callbacks=[checkpoint])

8064
Epoch 1/10
126/126 [==============================] - ETA: 0s - loss: 1.0791 - accuracy: 0.3991
Epoch 00001: val_accuracy improved from -inf to 0.35015, saving model to temp.hdf5
126/126 [==============================] - 59s 470ms/step - loss: 1.0791 - accuracy: 0.3991 - val_loss: 1.0938 - val_accuracy: 0.3501
Epoch 2/10
126/126 [==============================] - ETA: 0s - loss: 1.0237 - accuracy: 0.4704
Epoch 00002: val_accuracy improved from 0.35015 to 0.37793, saving model to temp.hdf5
126/126 [==============================] - 59s 467ms/step - loss: 1.0237 - accuracy: 0.4704 - val_loss: 1.0919 - val_accuracy: 0.3779
Epoch 3/10
126/126 [==============================] - ETA: 0s - loss: 0.9738 - accuracy: 0.5066
Epoch 00003: val_accuracy improved from 0.37793 to 0.38989, saving model to temp.hdf5
126/126 [==============================] - 59s 467ms/step - loss: 0.9738 - accuracy: 0.5066 - val_loss: 1.0827 - val_accuracy: 0.3899
Epoch 4/10
126/126 [==============================

In [15]:
model.load_weights(filepath)
yhat_te = model.predict(X_te)

In [16]:
wrong = np.argmax(yhat_te,axis=1) != np.argmax(y_te,axis=1)
d = Counter()
for t in np.sort(np.argmax(y_te,axis=1)[wrong]):
    d[t] += 1
dh = Counter()
for t in np.sort(np.argmax(yhat_te,axis=1)[wrong]):
    dh[t] += 1
d,dh,np.mean(1-wrong)

(Counter({0: 1709, 1: 2280, 2: 596}),
 Counter({0: 834, 1: 354, 2: 3397}),
 0.42887394120577976)

In [17]:
scores = model.evaluate(X_te, y_te, verbose=0)
'Test accuracy:', scores[1]

('Test accuracy:', 0.4288739264011383)

In [18]:
covid = np.zeros(len(_X_te), dtype=np.bool)
for i,x in enumerate(_X_te):
    hascovid = False
    for w in x:
        if ('covid' in w) or (w == 'coronavirus') or (w == 'corona') or (w == 'rona'):
            hascovid = True
    if hascovid:
        covid[i] = True
yhat_te[covid * wrong],y_te[covid * wrong],1-sum(covid * wrong)/sum(covid)

(array([[0.15300566, 0.49906442, 0.34792992],
        [0.3013195 , 0.12431338, 0.57436705],
        [0.31626573, 0.46778116, 0.21595317],
        [0.4334097 , 0.44681674, 0.11977359],
        [0.22137852, 0.31772402, 0.46089745],
        [0.08621107, 0.10044273, 0.8133462 ],
        [0.7441522 , 0.01730458, 0.23854329],
        [0.55256987, 0.00156088, 0.44586933],
        [0.01264207, 0.0052326 , 0.9821253 ],
        [0.0708338 , 0.08925496, 0.8399113 ],
        [0.51129454, 0.0571691 , 0.43153635],
        [0.384753  , 0.5554851 , 0.05976187],
        [0.3650018 , 0.3507774 , 0.28422078],
        [0.30153412, 0.10372447, 0.59474146],
        [0.004078  , 0.09007261, 0.90584934],
        [0.40151992, 0.00909945, 0.5893807 ],
        [0.09336989, 0.03027786, 0.8763522 ],
        [0.32970646, 0.19888455, 0.471409  ],
        [0.43604702, 0.13704975, 0.42690328],
        [0.26174808, 0.00955828, 0.72869366],
        [0.5223157 , 0.02340763, 0.45427665],
        [0.16893142, 0.3382739 , 0